In [5]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

from pprint import pprint
import collections
import re
import string
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
df = pd.read_csv("labeled_total.csv", keep_default_na=False)

In [7]:
print(df.shape)
df['lab_final'].value_counts()

(3583, 9)


NON_D_VIOLENCE_ABOUT       1905
NO_VIOLENCE                 954
NON_D_VIOLENCE_DIRECTED     464
DV_OPINION_ADVOCATE         149
DV_OPINION_INFO_NEWS         53
DV_STORY                     40
DV_OPINION_DENIER            18
Name: lab_final, dtype: int64

## Making a new Label Column
Combining Non_DV and DV labels to form fewer classes

In [0]:
df['new_label'] = df['lab_final']

In [0]:
df['new_label'].replace("DV_OPINION_DENIER",value="DV",inplace=True)
df['new_label'].replace("DV_OPINION_ADVOCATE",value="DV",inplace=True)
df['new_label'].replace("DV_OPINION_INFO_NEWS",value="DV",inplace=True)
df['new_label'].replace("NON_D_VIOLENCE_ABOUT",value="Harassment",inplace=True)
df['new_label'].replace("NON_D_VIOLENCE_DIRECTED",value="Harassment",inplace=True)

In [10]:
df.new_label.value_counts()

Harassment     2369
NO_VIOLENCE     954
DV              220
DV_STORY         40
Name: new_label, dtype: int64

In [0]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from tqdm import tqdm

## Using Multilingual Embedding

In [12]:
# Use the following statement to install tensorflow-text otherwise the embedding won't load
!pip install tensorflow-text==2.0.0 --user

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [0]:
# use df.lab_final for original labels
train, test, y_train, y_test = train_test_split(df.text, 
                                                df.new_label, 
                                                test_size=0.15, 
                                                random_state=42, 
                                                stratify=df.new_label)

In [0]:
embedding = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

In [0]:
def get_embedding(sen):
  encoding = embedding(sen)
  proper_encoding = tf.reshape(encoding, [-1]).numpy()
  return proper_encoding

In [17]:
x_train = []
for sentence in tqdm(train.values):
    x_train.append(get_embedding(sentence))

x_train = np.array(x_train)


x_test = []
for sentence in tqdm(test.values):
    x_test.append(get_embedding(sentence))

x_test = np.array(x_test)

100%|██████████| 538/538 [00:43<00:00, 12.45it/s]


In [18]:
len(x_train)

3045

## Modeling starts :

In [0]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, recall_score, precision_score
from sklearn.metrics import matthews_corrcoef

In [0]:
def evaluate_model(y_test, predicted_test):
    print ('Accuracy:', accuracy_score(y_test, predicted_test))
    print ('F1 score:', f1_score(y_test, predicted_test,average='weighted'))
    print ('Recall:', recall_score(y_test, predicted_test,average='weighted'))
    print ('Precision:', precision_score(y_test, predicted_test,average='weighted'))
    print('MCC: ',matthews_corrcoef(y_test, predicted_test))
    print ('\n Classification Report:\n', classification_report(y_test, predicted_test))

### Oversampling - SKIP
Didn't work well hence not using as of now <br />
Can be skipped if deemed unfit. Try Undersampling

In [0]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler

In [0]:
sm = SMOTE(random_state=42,k_neighbors=2,sampling_strategy='minority')
x_train, y_train = sm.fit_resample(x_train, y_train)

In [0]:
ros = RandomOverSampler(random_state=42,sampling_strategy='minority')
x_train, y_train = ros.fit_resample(x_train, y_train)

In [0]:
sm.get_params()

{'k_neighbors': 2,
 'kind': 'deprecated',
 'm_neighbors': 'deprecated',
 'n_jobs': 1,
 'out_step': 'deprecated',
 'random_state': 42,
 'ratio': None,
 'sampling_strategy': 'minority',
 'svm_estimator': 'deprecated'}

## Training Phase Starts :

In [0]:
from sklearn.model_selection import GridSearchCV

In [0]:
def param_selection(classifier, param_grid, X, y, nfolds):
  grid_search = GridSearchCV(classifier, param_grid, cv=nfolds,scoring='f1_weighted')
  grid_search.fit(X, y)
  grid_search.best_params_
  return grid_search

In [0]:
from sklearn.ensemble import RandomForestClassifier
param_grid = { 
    'n_estimators': [10, 30, 60, 80, 100, 120, 150]
}
# 'n_estimators': [100, 200, 300, 400, 500]
#'max_depth' : [4,5,6,7,8]
#'max_features': ['auto', 'sqrt', 'log2'],
#'criterion' :['gini', 'entropy']

model = param_selection(RandomForestClassifier(random_state=42, class_weight='balanced'), param_grid, x_train, y_train, 5)
print(model.best_params_)
predicted_test = model.predict(x_test)
evaluate_model(y_test, predicted_test)

#labels = ['NON_D_VIOLENCE_ABOUT', 'NON_D_VIOLENCE_DIRECTED','DV_OPINION_ADVOCATE','DV_OPINION_INFO_NEWS','NO_VIOLENCE','DV_OPINION_DENIER','DV_STORY'] 
labels = ['DV','DV_STORY','Harassment','NO_VIOLENCE'] 
cm = confusion_matrix(y_test, predicted_test,labels)
print('Confusion Matrix:')
print(labels)
print(cm)

{'n_estimators': 30}
Accuracy: 0.741635687732342
F1 score: 0.6898455670233717
Recall: 0.741635687732342
Precision: 0.7721500777894129
MCC:  0.4044243053528557

 Classification Report:
               precision    recall  f1-score   support

          DV       1.00      0.12      0.22        33
    DV_STORY       0.00      0.00      0.00         6
  Harassment       0.73      0.98      0.83       356
 NO_VIOLENCE       0.87      0.32      0.47       143

    accuracy                           0.74       538
   macro avg       0.65      0.36      0.38       538
weighted avg       0.77      0.74      0.69       538

Confusion Matrix:
['DV', 'DV_STORY', 'Harassment', 'NO_VIOLENCE']
[[  4   0  29   0]
 [  0   0   6   0]
 [  0   0 349   7]
 [  0   0  97  46]]


In [0]:
from sklearn.svm import SVC
param_grid = {'C': [2.6,3,3.1,3.2,3.3,3.4]}
#[0.2,0.6,1,1.2,1.5,1.8,2,2.2,2.4,2.6]
model = param_selection(SVC(kernel='linear'), param_grid, x_train, y_train, 5)
print(model.best_params_)
predicted_test = model.predict(x_test)
evaluate_model(y_test, predicted_test)
#labels = ['NON_D_VIOLENCE_ABOUT', 'NON_D_VIOLENCE_DIRECTED','DV_OPINION_ADVOCATE','DV_OPINION_INFO_NEWS','NO_VIOLENCE','DV_OPINION_DENIER','DV_STORY'] 
labels = ['DV','DV_STORY','Harassment','NO_VIOLENCE'] 
cm = confusion_matrix(y_test, predicted_test,labels)
print('Confusion Matrix:')
print(labels)
print(cm)

{'C': 3.2}
Accuracy: 0.8122676579925651
F1 score: 0.801048916092298
Recall: 0.8122676579925651
Precision: 0.8084766121737972
MCC:  0.5890163336785457

 Classification Report:
               precision    recall  f1-score   support

          DV       0.83      0.45      0.59        33
    DV_STORY       0.50      0.17      0.25         6
  Harassment       0.82      0.93      0.87       356
 NO_VIOLENCE       0.79      0.62      0.70       143

    accuracy                           0.81       538
   macro avg       0.74      0.54      0.60       538
weighted avg       0.81      0.81      0.80       538

Confusion Matrix:
['DV', 'DV_STORY', 'Harassment', 'NO_VIOLENCE']
[[ 15   0  17   1]
 [  0   1   5   0]
 [  1   0 332  23]
 [  2   1  51  89]]


In [0]:
param_grid = {'C': [1.07], 'gamma' : [2.075]}
model = param_selection(SVC(kernel='rbf'), param_grid, x_train, y_train, 5)
print(model.best_params_)
predicted_test = model.predict(x_test)
evaluate_model(y_test, predicted_test)
#labels = ['NON_D_VIOLENCE_ABOUT', 'NON_D_VIOLENCE_DIRECTED','DV_OPINION_ADVOCATE','DV_OPINION_INFO_NEWS','NO_VIOLENCE','DV_OPINION_DENIER','DV_STORY'] 
labels = ['DV','DV_STORY','Harassment','NO_VIOLENCE'] 
cm = confusion_matrix(y_test, predicted_test,labels)
print('Confusion Matrix:')
print(labels)
print(cm)

{'C': 1.07, 'gamma': 2.075}
Accuracy: 0.8104089219330854
F1 score: 0.790477334875256
Recall: 0.8104089219330854
Precision: 0.8122642523696325
MCC:  0.5818659680131072

 Classification Report:
               precision    recall  f1-score   support

          DV       0.92      0.36      0.52        33
    DV_STORY       0.00      0.00      0.00         6
  Harassment       0.80      0.96      0.87       356
 NO_VIOLENCE       0.86      0.57      0.68       143

    accuracy                           0.81       538
   macro avg       0.65      0.47      0.52       538
weighted avg       0.81      0.81      0.79       538

Confusion Matrix:
['DV', 'DV_STORY', 'Harassment', 'NO_VIOLENCE']
[[ 12   0  21   0]
 [  0   0   6   0]
 [  0   0 343  13]
 [  1   0  61  81]]


In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

In [0]:
param_grid  = {'C': [2.4,2.5,2.6,2.7,2.8,2.9,3]}
#[0.2,0.6,1,1.2,1.5,1.8,1.9,2]
model = param_selection(LogisticRegression(class_weight='balanced'), param_grid, x_train,y_train, 5)
print(model.best_params_)
predicted_test=model.predict(x_test)
evaluate_model(y_test, predicted_test)
#labels = ['NON_D_VIOLENCE_ABOUT', 'NON_D_VIOLENCE_DIRECTED','DV_OPINION_ADVOCATE','DV_OPINION_INFO_NEWS','NO_VIOLENCE','DV_OPINION_DENIER','DV_STORY'] 
labels = ['DV','DV_STORY','Harassment','NO_VIOLENCE'] 
cm = confusion_matrix(y_test, predicted_test,labels)
print('Confusion Matrix:')
print(labels)
print(cm)

{'C': 2.8}
Accuracy: 0.7546468401486989
F1 score: 0.7633282202361663
Recall: 0.7546468401486989
Precision: 0.7818985288414716
MCC:  0.5411627518202625

 Classification Report:
               precision    recall  f1-score   support

          DV       0.50      0.76      0.60        33
    DV_STORY       0.23      0.50      0.32         6
  Harassment       0.87      0.77      0.81       356
 NO_VIOLENCE       0.65      0.73      0.69       143

    accuracy                           0.75       538
   macro avg       0.56      0.69      0.61       538
weighted avg       0.78      0.75      0.76       538

Confusion Matrix:
['DV', 'DV_STORY', 'Harassment', 'NO_VIOLENCE']
[[ 25   1   5   2]
 [  0   3   3   0]
 [ 21   8 273  54]
 [  4   1  33 105]]


In [0]:
param_grid = {'max_depth' : range(1,33)}
#'max_depth' : range(4,33)
model = param_selection(DecisionTreeClassifier(class_weight='balanced', random_state=0), param_grid, x_train, y_train, 5)
print(model.best_params_)
predicted_test=model.predict(x_test)
evaluate_model(y_test, predicted_test)
#labels = ['NON_D_VIOLENCE_ABOUT', 'NON_D_VIOLENCE_DIRECTED','DV_OPINION_ADVOCATE','DV_OPINION_INFO_NEWS','NO_VIOLENCE','DV_OPINION_DENIER','DV_STORY'] 
labels = ['DV','DV_STORY','Harassment','NO_VIOLENCE'] 
cm = confusion_matrix(y_test, predicted_test,labels)
print('Confusion Matrix:')
print(labels)
print(cm)

{'max_depth': 13}
Accuracy: 0.654275092936803
F1 score: 0.6625410251292412
Recall: 0.654275092936803
Precision: 0.6736865419298866
MCC:  0.3221137103659988

 Classification Report:
               precision    recall  f1-score   support

          DV       0.25      0.39      0.30        33
    DV_STORY       0.33      0.33      0.33         6
  Harassment       0.78      0.74      0.76       356
 NO_VIOLENCE       0.52      0.52      0.52       143

    accuracy                           0.65       538
   macro avg       0.47      0.50      0.48       538
weighted avg       0.67      0.65      0.66       538

Confusion Matrix:
['DV', 'DV_STORY', 'Harassment', 'NO_VIOLENCE']
[[ 13   2  13   5]
 [  2   2   1   1]
 [ 29   2 263  62]
 [  9   0  60  74]]


In [0]:
param_grid = {'n_neighbors' : range(2,20)}
model = param_selection(KNeighborsClassifier(weights='distance') , param_grid, x_train, y_train, 5)
print(model.best_params_)
predicted_test=model.predict(x_test)
evaluate_model(y_test, predicted_test)
#labels = ['NON_D_VIOLENCE_ABOUT', 'NON_D_VIOLENCE_DIRECTED','DV_OPINION_ADVOCATE','DV_OPINION_INFO_NEWS','NO_VIOLENCE','DV_OPINION_DENIER','DV_STORY'] 
labels = ['DV','DV_STORY','Harassment','NO_VIOLENCE'] 
cm = confusion_matrix(y_test, predicted_test,labels)
print('Confusion Matrix:')
print(labels)
print(cm)

{'n_neighbors': 5}
Accuracy: 0.7620817843866171
F1 score: 0.7444501035356337
Recall: 0.7620817843866171
Precision: 0.755070297221586
MCC:  0.47214569598329065

 Classification Report:
               precision    recall  f1-score   support

          DV       0.64      0.48      0.55        33
    DV_STORY       0.33      0.17      0.22         6
  Harassment       0.77      0.92      0.84       356
 NO_VIOLENCE       0.75      0.46      0.57       143

    accuracy                           0.76       538
   macro avg       0.62      0.51      0.55       538
weighted avg       0.76      0.76      0.74       538

Confusion Matrix:
['DV', 'DV_STORY', 'Harassment', 'NO_VIOLENCE']
[[ 16   1  16   0]
 [  1   1   4   0]
 [  6   1 327  22]
 [  2   0  75  66]]


In [0]:
# use .toarray() with tifidf vectors not with other embeddings
param_grid = {}
model = param_selection(GaussianNB() , param_grid, x_train,y_train, 5)
print(model.best_params_)
predicted_test=model.predict(x_test)
evaluate_model(y_test, predicted_test)
#labels = ['NON_D_VIOLENCE_ABOUT', 'NON_D_VIOLENCE_DIRECTED','DV_OPINION_ADVOCATE','DV_OPINION_INFO_NEWS','NO_VIOLENCE','DV_OPINION_DENIER','DV_STORY'] 
labels = ['DV','DV_STORY','Harassment','NO_VIOLENCE'] 
cm = confusion_matrix(y_test, predicted_test,labels)
print('Confusion Matrix:')
print(labels)
print(cm)

{}
Accuracy: 0.6821561338289963
F1 score: 0.6989654597306801
Recall: 0.6821561338289963
Precision: 0.763085210023418
MCC:  0.4779068651210925

 Classification Report:
               precision    recall  f1-score   support

          DV       0.28      0.64      0.39        33
    DV_STORY       0.33      0.67      0.44         6
  Harassment       0.88      0.63      0.73       356
 NO_VIOLENCE       0.60      0.83      0.69       143

    accuracy                           0.68       538
   macro avg       0.52      0.69      0.57       538
weighted avg       0.76      0.68      0.70       538

Confusion Matrix:
['DV', 'DV_STORY', 'Harassment', 'NO_VIOLENCE']
[[ 21   2   8   2]
 [  0   4   2   0]
 [ 48   6 224  78]
 [  5   0  20 118]]


In [25]:
# Doesn't work with universal embedding because :
# ValueError: Negative values in data passed to MultinomialNB (input X)
'''from sklearn.naive_bayes import MultinomialNB
param_grid = {}
model = param_selection(MultinomialNB() , param_grid, x_train,y_train, 5)
print(model.best_params_)
predicted_test=model.predict(x_test)
evaluate_model(y_test, predicted_test)
#labels = ['NON_D_VIOLENCE_ABOUT', 'NON_D_VIOLENCE_DIRECTED','DV_OPINION_ADVOCATE','DV_OPINION_INFO_NEWS','NO_VIOLENCE','DV_OPINION_DENIER','DV_STORY'] 
labels = ['DV','DV_STORY','Harassment','NO_VIOLENCE'] 
cm = confusion_matrix(y_test, predicted_test,labels)
print('Confusion Matrix:')
print(labels)
print(cm)'''

"from sklearn.naive_bayes import MultinomialNB\nparam_grid = {}\nmodel = param_selection(MultinomialNB() , param_grid, x_train,y_train, 5)\nprint(model.best_params_)\npredicted_test=model.predict(x_test)\nevaluate_model(y_test, predicted_test)\n#labels = ['NON_D_VIOLENCE_ABOUT', 'NON_D_VIOLENCE_DIRECTED','DV_OPINION_ADVOCATE','DV_OPINION_INFO_NEWS','NO_VIOLENCE','DV_OPINION_DENIER','DV_STORY'] \nlabels = ['DV','DV_STORY','Harassment','NO_VIOLENCE'] \ncm = confusion_matrix(y_test, predicted_test,labels)\nprint('Confusion Matrix:')\nprint(labels)\nprint(cm)"

## Ensemble Methods - Voting

In [0]:
from sklearn.ensemble import VotingClassifier
# try both soft and hard voting
model=VotingClassifier(estimators=[('KNN',KNeighborsClassifier(n_neighbors=5)),
                                   ('LinSVM',SVC(kernel='linear',C=3.2,probability=True)),
                                   ('RBF',SVC(kernel='rbf',C=1.07,gamma=2.075,probability=True))
                                   ], 
                       voting='soft').fit(x_train,y_train)
#('LR',LogisticRegression(C=2.8))
#('RBF',SVC(kernel='rbf',C=1.07,gamma=2.075,probability=True)),
#('RFor',RandomForestClassifier(n_estimators=30,random_state=0)),
#('DT',DecisionTreeClassifier(max_depth=13)),
#('NB',GaussianNB())
predicted_test=model.predict(x_test)
evaluate_model(y_test, predicted_test)
#labels = ['NON_D_VIOLENCE_ABOUT', 'NON_D_VIOLENCE_DIRECTED','DV_OPINION_ADVOCATE','DV_OPINION_INFO_NEWS','NO_VIOLENCE','DV_OPINION_DENIER','DV_STORY'] 
labels = ['DV','DV_STORY','Harassment','NO_VIOLENCE'] 
cm = confusion_matrix(y_test, predicted_test,labels)
print('Confusion Matrix:')
print(labels)
print(cm)

Accuracy: 0.8234200743494424
F1 score: 0.8099742686483742
Recall: 0.8234200743494424
Precision: 0.8283019458689672
MCC:  0.6164193027197981

 Classification Report:
               precision    recall  f1-score   support

          DV       0.89      0.52      0.65        33
    DV_STORY       0.33      0.17      0.22         6
  Harassment       0.81      0.96      0.88       356
 NO_VIOLENCE       0.87      0.57      0.69       143

    accuracy                           0.82       538
   macro avg       0.73      0.55      0.61       538
weighted avg       0.83      0.82      0.81       538

Confusion Matrix:
['DV', 'DV_STORY', 'Harassment', 'NO_VIOLENCE']
[[ 17   1  15   0]
 [  1   1   4   0]
 [  0   1 343  12]
 [  1   0  60  82]]


In [28]:
from sklearn.ensemble import BaggingClassifier
param_grid = {'n_estimators' : [300], 'base_estimator__n_neighbors' : [5]}
#param_grid = {}
model = param_selection(BaggingClassifier(base_estimator=KNeighborsClassifier(),random_state=0), param_grid, x_train, y_train, 5)
print(model.best_params_)
predicted_test=model.predict(x_test)
evaluate_model(y_test, predicted_test)
#labels = ['NON_D_VIOLENCE_ABOUT', 'NON_D_VIOLENCE_DIRECTED','DV_OPINION_ADVOCATE','DV_OPINION_INFO_NEWS','NO_VIOLENCE','DV_OPINION_DENIER','DV_STORY'] 
labels = ['DV','DV_STORY','Harassment','NO_VIOLENCE'] 
cm = confusion_matrix(y_test, predicted_test,labels)
print('Confusion Matrix:')
print(labels)
print(cm)

{'base_estimator__n_neighbors': 5, 'n_estimators': 300}
Accuracy: 0.7695167286245354
F1 score: 0.7494216900988511
Recall: 0.7695167286245354
Precision: 0.7669381870776933
MCC:  0.48638193027548093

 Classification Report:
               precision    recall  f1-score   support

          DV       0.71      0.45      0.56        33
    DV_STORY       0.33      0.17      0.22         6
  Harassment       0.77      0.94      0.85       356
 NO_VIOLENCE       0.78      0.45      0.58       143

    accuracy                           0.77       538
   macro avg       0.65      0.50      0.55       538
weighted avg       0.77      0.77      0.75       538

Confusion Matrix:
['DV', 'DV_STORY', 'Harassment', 'NO_VIOLENCE']
[[ 15   1  17   0]
 [  1   1   4   0]
 [  4   1 333  18]
 [  1   0  77  65]]


In [36]:
param_grid = {'n_estimators' : [300],'base_estimator__max_depth':[13]}
#param_grid = {}
model = param_selection(BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight='balanced'),random_state=0), param_grid, x_train, y_train, 5)
print(model.best_params_)
predicted_test=model.predict(x_test)
evaluate_model(y_test, predicted_test)
#labels = ['NON_D_VIOLENCE_ABOUT', 'NON_D_VIOLENCE_DIRECTED','DV_OPINION_ADVOCATE','DV_OPINION_INFO_NEWS','NO_VIOLENCE','DV_OPINION_DENIER','DV_STORY'] 
labels = ['DV','DV_STORY','Harassment','NO_VIOLENCE'] 
cm = confusion_matrix(y_test, predicted_test,labels)
print('Confusion Matrix:')
print(labels)
print(cm)

{'base_estimator__max_depth': 13, 'n_estimators': 300}
Accuracy: 0.7769516728624535
F1 score: 0.7477325074685431
Recall: 0.7769516728624535
Precision: 0.7882119946047307
MCC:  0.4995910273702181

 Classification Report:
               precision    recall  f1-score   support

          DV       1.00      0.33      0.50        33
    DV_STORY       0.00      0.00      0.00         6
  Harassment       0.76      0.97      0.85       356
 NO_VIOLENCE       0.84      0.44      0.58       143

    accuracy                           0.78       538
   macro avg       0.65      0.44      0.48       538
weighted avg       0.79      0.78      0.75       538

Confusion Matrix:
['DV', 'DV_STORY', 'Harassment', 'NO_VIOLENCE']
[[ 11   0  22   0]
 [  0   0   6   0]
 [  0   0 344  12]
 [  0   0  80  63]]


In [34]:
from sklearn.ensemble import AdaBoostClassifier
param_grid = {'n_estimators' : [300]}
model = param_selection(AdaBoostClassifier(random_state=0), param_grid, x_train, y_train, 5)
print(model.best_params_)
predicted_test=model.predict(x_test)
evaluate_model(y_test, predicted_test)
#labels = ['NON_D_VIOLENCE_ABOUT', 'NON_D_VIOLENCE_DIRECTED','DV_OPINION_ADVOCATE','DV_OPINION_INFO_NEWS','NO_VIOLENCE','DV_OPINION_DENIER','DV_STORY'] 
labels = ['DV','DV_STORY','Harassment','NO_VIOLENCE'] 
cm = confusion_matrix(y_test, predicted_test,labels)
print('Confusion Matrix:')
print(labels)
print(cm)

{'n_estimators': 300}
Accuracy: 0.6561338289962825
F1 score: 0.6620501363011688
Recall: 0.6561338289962825
Precision: 0.6876833677302117
MCC:  0.3556409661429277

 Classification Report:
               precision    recall  f1-score   support

          DV       0.54      0.45      0.49        33
    DV_STORY       0.00      0.00      0.00         6
  Harassment       0.80      0.67      0.73       356
 NO_VIOLENCE       0.47      0.69      0.56       143

    accuracy                           0.66       538
   macro avg       0.45      0.45      0.45       538
weighted avg       0.69      0.66      0.66       538

Confusion Matrix:
['DV', 'DV_STORY', 'Harassment', 'NO_VIOLENCE']
[[ 15   0  14   4]
 [  1   0   5   0]
 [  8   0 239 109]
 [  4   0  40  99]]


In [32]:
from sklearn.ensemble import GradientBoostingClassifier
param_grid = {'n_estimators':[300]}
model = param_selection(GradientBoostingClassifier(random_state=0), param_grid, x_train, y_train, 5)
print(model.best_params_)
predicted_test=model.predict(x_test)
evaluate_model(y_test, predicted_test)
#labels = ['NON_D_VIOLENCE_ABOUT', 'NON_D_VIOLENCE_DIRECTED','DV_OPINION_ADVOCATE','DV_OPINION_INFO_NEWS','NO_VIOLENCE','DV_OPINION_DENIER','DV_STORY'] 
labels = ['DV','DV_STORY','Harassment','NO_VIOLENCE'] 
cm = confusion_matrix(y_test, predicted_test,labels)
print('Confusion Matrix:')
print(labels)
print(cm)

{'n_estimators': 300}
Accuracy: 0.8029739776951673
F1 score: 0.7921881560816727
Recall: 0.8029739776951673
Precision: 0.8041872657585506
MCC:  0.5680117008824691

 Classification Report:
               precision    recall  f1-score   support

          DV       0.94      0.48      0.64        33
    DV_STORY       0.00      0.00      0.00         6
  Harassment       0.81      0.93      0.87       356
 NO_VIOLENCE       0.79      0.59      0.67       143

    accuracy                           0.80       538
   macro avg       0.64      0.50      0.55       538
weighted avg       0.80      0.80      0.79       538

Confusion Matrix:
['DV', 'DV_STORY', 'Harassment', 'NO_VIOLENCE']
[[ 16   0  16   1]
 [  0   0   6   0]
 [  0   3 332  21]
 [  1   2  56  84]]


In [33]:
import xgboost as xg
param_grid = {'n_estimators':[300]}
model = param_selection(xg.XGBClassifier(), param_grid, x_train, y_train, 5)
print(model.best_params_)
predicted_test=model.predict(x_test)
evaluate_model(y_test, predicted_test)
#labels = ['NON_D_VIOLENCE_ABOUT', 'NON_D_VIOLENCE_DIRECTED','DV_OPINION_ADVOCATE','DV_OPINION_INFO_NEWS','NO_VIOLENCE','DV_OPINION_DENIER','DV_STORY'] 
labels = ['DV','DV_STORY','Harassment','NO_VIOLENCE'] 
cm = confusion_matrix(y_test, predicted_test,labels)
print('Confusion Matrix:')
print(labels)
print(cm)

{'n_estimators': 300}
Accuracy: 0.8141263940520446
F1 score: 0.7987007691912357
Recall: 0.8141263940520446
Precision: 0.8104136190058175
MCC:  0.5910930565014488

 Classification Report:
               precision    recall  f1-score   support

          DV       0.94      0.48      0.64        33
    DV_STORY       0.00      0.00      0.00         6
  Harassment       0.81      0.95      0.87       356
 NO_VIOLENCE       0.82      0.59      0.69       143

    accuracy                           0.81       538
   macro avg       0.64      0.51      0.55       538
weighted avg       0.81      0.81      0.80       538

Confusion Matrix:
['DV', 'DV_STORY', 'Harassment', 'NO_VIOLENCE']
[[ 16   0  16   1]
 [  0   0   6   0]
 [  1   0 338  17]
 [  0   0  59  84]]
